# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A **credit scoring** is used to evaluate the ability of a potential borrower to repay their loan.

In [1]:
# Libraries and project environment setup:

## pandas and numpy for data preprocessing and manipulation
import pandas as pd
import numpy as np

## WordNet Lemmatizer and SnowballStemmer for handling categorical variables
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.stem import SnowballStemmer
english_stemmer = SnowballStemmer('english')

##Remove warnings
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore");

## Open the data file and have a look at the general information. 

In [2]:
# Csv file read
## Use try-except to handle unexpected errors
try:
    credit = pd.read_csv('/datasets/credit_scoring_eng.csv')
except: 
    credit = pd.read_csv('https://code.s3.yandex.net/datasets/credit_scoring_eng.csv')
credit

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


Reading the file we immediately find errors when it comes to **days_employed** (can't be negative values)

In [3]:
# General file examination
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Results show column names are accurate to the project data description.

There are 3 data types: float, object and integer.

The dataset includes 21525 entries and 12 columns.

We can clearly see that **days_employed** and **total_income** columns contain missing values.

5 columns are categorical, 6 are quantical, and **debt** contains quantical labels rather than quantical values.

In [4]:
# Integreted display of the quantical columns
credit.describe(include='all')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,19351.000000,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,secondary education,NaN,married,NaN,F,employee,NaN,NaN,wedding ceremony
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,26787.568355,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,16475.450632,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,3306.762000,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,16488.504500,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,23202.870000,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,32549.611000,NaN


Output shows minimum value in **children** column -1 and maximum of 20, -1 is a distinct error, whereas 20 seems too high and must be further investigated.

**dob_years** column min is 0, an error that needs to be fixed.

Mean value of **debt column** is closer to 0 than 1 meaning most people in the dataset repay their loan.

### Conclusion

During this first phase we setup the project environment, open the dataset, and initially examine the data.

The dataset contains some errors and missing data, which need to be fixed, for example:

**children** quantical. Min value is -1 and max 20, unlikely values.

**day_employed**  quantical. Contains negative values, nulls and max of 401755 which is aprox 1110 years, an impossible figure.

**total_income**  quantical. Contains nulls.

**dob_years**  quantical. Min value is 0, an error value for age, most likely need to be over 18 years old to take/get a bank loan.

**family_status** categorical and **family_status_id**  quantical. Both provide us with the same information.

**gender** categorical. Contains 3 possible values, errors need to be fixed.

**income_type**  categorical. Seems ok.

**education** categorical and **education_id**  quantical values per category. Technically provide the same information.

**debt**  quantical. Indicates the borrower's ability to repay their loan, only contains integer values 0 and 1 which seems ok.

**purpose** categorical variable which seem ok.

## Data preprocessing

In this second phase we start by identifying the missing data and evaluate their impact on the dataset data reliability in order to decide whether to fill in or drop these entries.

Additionally, we'll find errors and correct them.

Next, replace the data type of all floats to integers in order to make further processing easier.

Then, drop duplicated entries and finally, categorize and create a new table for future analysis.

### Processing missing values

Dealing with missing values includes the following procedure:
- We determine if there are missing values.
- We count how many missing values we need to deal with.
- We determine how significant are the missing values and if below 5-10% we drop them.
- We determine if missing values belong to categorical or quantitative variables, and handle them accordingly.

In [5]:
#Before we start with missing values, 
#we look at the percentage of missing values in columns,
#not simply the amount of missing values per column:
credit.isna().sum()*100/len(credit)
pd.DataFrame(round((credit.isna().mean()*100),2)).style.background_gradient('coolwarm')
credit.isna().mean() * 100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

In [6]:
# Here we start by identifying if there are any missing values using if-else.
if credit.isna().values.any() == True:
    print(credit.isna().sum())
else:
    print('No NA in dataset')

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Results show that **days_employed** and **total_income** contain nulls. Since the same amount of missing values was identified, we am curious if these are both in the same entries.

In [7]:
#Here we check the the percentage of the missing values of days_employed in the dataset.
#(credit['days_employed'].isnull().sum() / len(credit)) * 100

A rule of thumb is that we don't drop more than 5%-10% of the data. Since results show that it's just over 10% by a bit, we decide to fill-in and not drop any data.

In [8]:
#Here we replace null enteris of days_empoyed with the median of the dataset values for this column. 
#We choose median rather than mean.
#We believe it is safer to reflect as such the data distribution.
#Negative values are assumed to be correct in their abs value.
credit['days_employed'] = credit['days_employed'].abs()
credit['days_employed'].fillna(credit['days_employed'].median(), inplace=True)
credit.isnull().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

As we see, the dataset entries of **days_employed** are no longer missing. 

In [9]:
#Now, we check the the percentage of the missing values of total_income in the dataset.
#(credit['total_income'].isnull().sum() / len(credit)) * 100

Once again, following the rule of thumb we don't drop more than 5%-10% of the data. Since results for **total_income** are just over 10% by a bit, we decide to fill-in and not drop any data.

In [10]:
#Here we group the values based on education and income_type and the replace the missing values with the medium calculated for each result and group.
credit['total_income'] = credit['total_income'].fillna(credit.groupby(['education','income_type'])['total_income'].transform('median'))
credit.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Result show that the dataset entries of **total_income** are no longer missing. 

In conclusion, we see that the dataset is complete with no missing values.
The possible causes for the missing values are due to human error or technical malfunction of the bank system.  

Next, we will check for data errors.

In [11]:
#We check the values of number of children. Obviously it can't be a negative number or a large unlikely number.
credit['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Results show we entries with a negative value (-1) and 76 entries with 20 children which we believe is unlikely a true value.

In [12]:
#First, we change negative values to positive using abs function.
#Next, we replace value of 20 children with the median value.
credit['children'] = credit['children'].abs()
credit.loc[credit['children'] == 20, 'children'] = credit['children'].median()
credit['children'].value_counts()

0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Nice! **children** column is now fixed.

In [13]:
#Moving on to fix days_employed.
credit['days_employed'].value_counts()

2194.220567      2175
329951.594147       1
886.253127          1
2539.534295         1
390574.985524       1
                 ... 
1320.841444         1
1394.302246         1
2325.720832         1
4086.407828         1
1636.419775         1
Name: days_employed, Length: 19351, dtype: int64

Negative numbers and extremely large values in **days_employed** are obviously errors.

In [14]:
#Negative values are assumed to be correct in their abs value.
#75 years is max in dataset, and we assume "official" employment starts at age 18.
#Based on the above, we calculate the days_employment as following
#credit['days_employed'] = credit['days_employed'].abs()
credit.loc[credit['days_employed'] > ((75-18) * 365), 'days_employed'] = credit['days_employed'].median()
credit['days_employed'].value_counts()

2194.220567    5620
142.276217        1
601.464943        1
3545.955468       1
622.217198        1
               ... 
6219.157426       1
5873.586442       1
3355.260838       1
472.196656        1
1636.419775       1
Name: days_employed, Length: 15906, dtype: int64

Results look good! Moving on...

In [15]:
#We check dob_years.
credit['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [16]:
#Next we replace age values of 0 to the median of dataset. 
credit.loc[credit['dob_years'] == 0, 'dob_years'] = credit['dob_years'].median()
credit['dob_years'].value_counts()

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [17]:
#Here we check education values.
credit['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [18]:
#We replaced all education column values to lower case to see if they are in sync with following id. 
#This we save us the need to do so later on when working on duplicats.
credit['education'] = credit['education'].str.lower()
credit['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

Perfect! **education** values in the dataset are unified.

In [19]:
#Here we check education_id values.
#There should be 5 just like in the education column distributed alike.
credit['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [20]:
#Given the eduction and education_id contain same information.
#We will use Dictionary to display education and education_id.
education_dict=pd.Series(credit.education.values, index=credit.education_id).to_dict()
education_dict

{0: "bachelor's degree",
 1: 'secondary education',
 2: 'some college',
 3: 'primary education',
 4: 'graduate degree'}

In [21]:
#Moving on to check family_status and family_status_id.
#Here we check family_status values.
credit['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

In [22]:
#Here we check family_status_id values.
#There should be 5 just like in the family_status column distributed alike.
credit['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [23]:
#Given the family_status and family_status_id contain same information.
#We will use Dictionary to display family_status and family_status_id.
family_status_dict=pd.Series(credit.family_status.values, index=credit.family_status_id).to_dict()
family_status_dict

{0: 'married',
 1: 'civil partnership',
 2: 'widow / widower',
 3: 'divorced',
 4: 'unmarried'}

Wonderful! We now have two dictionaries for future use.

In [24]:
#Here we check the values of gender.
credit['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [25]:
#Getting a closer look at the one entery with the strange gender value.
credit[credit['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


In [26]:
#Since there is only 1 entery with gender value of xna,
#we decide it is of low significance to the dataset balance and we drop it.
credit = credit[credit['gender'] != 'XNA']
credit['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

Interestingly, results show that female borrowers are almost twice the amount of male.

In [27]:
#Here we check income_type
credit['income_type'].value_counts()

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

**income_type** has a few outliers as shown above.

In [28]:
#Here we check debt
credit['debt'].value_counts()

0    19783
1     1741
Name: debt, dtype: int64

All good! **debt** consists of 0 and 1 values representing whether a customer has ever defaulted on a loan. 0=no debt and 1=debt.

In [29]:
#Here we check total_income
credit['total_income'].value_counts()

21890.8980    755
18317.3395    309
25544.4640    274
26586.2525    210
32305.8750    179
             ... 
26124.6130      1
28692.1820      1
28477.7830      1
26954.6080      1
41428.9160      1
Name: total_income, Length: 19369, dtype: int64

**total_income** has a few outliers as shown above.

In [30]:
#Finally, we check purpose of loan column values.
#To be on the safe side, we changed all values to lower case for data unification in advance.
credit['purpose'] = credit['purpose'].str.lower()
credit['purpose'].value_counts()

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
housing transactions                        653
buying property for renting out             653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             623
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

There are no missing values and no errors. We are ready to continue...

#### Conclusion

In this phase, we successfully completed the analysis of missing data and replacing the missing data with reasonable values as explained in each step of the process above.

### Data type replacement

In this section we shall convert the float data type to integers to make further analysis.

In [31]:
#First we find what colums contain what data type.
credit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     21524 non-null  float64
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  float64
 11  purpose           21524 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [32]:
#Based on info function results above, we can tell four columns are in float form.
#Childen, days_employed, dob_years and total_income logically should be integers.
#As such we convert these floats to integers. 
#We use the astype function as method to change the data type.
credit[['children','days_employed', 'dob_years', 'total_income']] = credit[['children','days_employed', 'dob_years', 'total_income']].astype(int)
credit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   days_employed     21524 non-null  int64 
 2   dob_years         21524 non-null  int64 
 3   education         21524 non-null  object
 4   education_id      21524 non-null  int64 
 5   family_status     21524 non-null  object
 6   family_status_id  21524 non-null  int64 
 7   gender            21524 non-null  object
 8   income_type       21524 non-null  object
 9   debt              21524 non-null  int64 
 10  total_income      21524 non-null  int64 
 11  purpose           21524 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Well done! All data types are correct. We can move on.

#### Conclusion

We successfully completed the data type replacement section, in which we converted all the float data type columns to integers making them suitable for further analysis.

### Processing duplicates

Moving on, we will process duplicates in the dataset.
We plan to unify the data into same formats for example:
- All **education** in lower case.
- **purpose** categorized into four main groups.
- Also we consider eliminating "duplicated" columns that provide same information such as **education_id** and **education**, as well as **family_status_id** and **family_status**.

In [33]:
#Here we sort all the values in purpose column into four main values.
#Practically, we catogorize these values.
#Doing this early on enables us to properly handle duplicates in the dataset.
for purpose in credit['purpose']:
    words = nltk.word_tokenize(purpose)
    if 'education'in words or 'university' in words or 'educated' in words:
        credit['purpose'].replace(purpose, 'education', inplace=True)
    if 'car' in words or 'cars'in words:
        credit['purpose'].replace(purpose, 'car', inplace=True)
    if 'house' in words or 'housing'in words or 'estate' in words or 'property' in words:
        credit['purpose'].replace(purpose, 'real estate', inplace=True)
    if 'wedding' in words:
        credit['purpose'].replace(purpose, 'wedding', inplace=True)
credit['purpose'].value_counts()

real estate    10839
car             4315
education       4022
wedding         2348
Name: purpose, dtype: int64

Outcome of this results show four main categories for **purpose** column.
This makes it much easier to further analyze the dataset entries. 
We see there a various purposes for the loan, some of which reflect same purpose in slightly different wording. We plan to take of it later on.

In [34]:
#After fixing the data for further analysis in previous phases,
#here we check for duplicate enteries in the dataset.
credit.duplicated().sum()

347

We found the above number of duplicates. Though 346 seems a large number, in relation to the overall number of entries in th dataset, it's insignificant and therefore we drop them.
We decide to remove duplicates only if the entire data entry is the same. Since there is no customer ID, we assume if all columns are alike then it is a definite duplicate.
We assume that the possible reasons for the appearance of duplicates in the data are human error and technical malfunction of the bank system.

In [35]:
#This is the actual droping process.
credit.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,real estate
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,real estate
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,education
4,0,2194,53,secondary education,1,civil partnership,1,F,retiree,0,25378,wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21172,1,4529,43,secondary education,1,civil partnership,1,F,business,0,35966,real estate
21173,0,2194,67,secondary education,1,married,0,F,retiree,0,24959,car
21174,1,2113,38,secondary education,1,civil partnership,1,M,employee,1,14347,real estate
21175,3,3112,38,secondary education,1,married,0,M,employee,1,39054,car


#### Conclusion

In conclusion, we see that dataset entries that were found duplicated were dropped. Following the project phases so far, now we are left with a unified and accurate dataset for further stages of analysis.

### Categorizing Data

In this section of the project we will categorize data and create the new data frame to only include columns relevant for the upcoming questions.

We follow the principle of data categorization for the relevant columns are **children**, **debt**, **family_status**, **income_type**, **total_income** and **purpose**.

Data categorization principles allow us to normalize and unify the data into distinct groups that are much easier to analyze, without losing any information relevant to the investigation at hand. 

In [36]:
#Here we group age into age groups
def dob_group(dob_years):
    if dob_years < 30:
        return '0-29'
    if dob_years < 40:
        return '30-39'
    if dob_years < 50:
        return '40-49'
    if dob_years < 60:
        return '50-59'
    return '60+'
credit['dob_years_group'] = credit['dob_years'].apply(dob_group)
credit['dob_years_group'].value_counts()

30-39    5674
40-49    5472
50-59    4679
0-29     3182
60+      2517
Name: dob_years_group, dtype: int64

Perfect! This makes data more easy to capture at a glimpse.  

In [37]:
#Here we check for outliers in the income_type column.
#We wish to to unite these enteries with others in order not to lose any additional data.
credit['income_type'].value_counts()

employee                       11119
business                        5084
retiree                         3856
civil servant                   1459
entrepreneur                       2
unemployed                         2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

In [38]:
#So here we combine unemployment, student and paternity / maternity leave with employee.
#employee to our undestaning is a persone who recieves some sort of monthly income.
#unemployed and paternity / maternity leave recieve wellfare benefits.
#entrepreneur will be combined with business. 
#student has a student job or parents monthly allowence support and therefore is employee.  
#Obviously, this change in 6 entries will impact the dataset variance insinificatly. 
credit.loc[credit['income_type'] == 'entrepreneur', 'income_type'] = 'business'
credit.loc[credit['income_type'] == 'unemployed', 'income_type'] = 'employee'  
credit.loc[credit['income_type'] == 'student', 'income_type'] = 'employee' 
credit.loc[credit['income_type'] == 'paternity / maternity leave', 'income_type'] = 'employee'
credit['income_type'].value_counts()

employee         11123
business          5086
retiree           3856
civil servant     1459
Name: income_type, dtype: int64

This result seems better for the purpose of our analysis.

In [39]:
#Next, we take a closer look at total_income values.
credit['total_income'].value_counts()

21890    755
18317    310
25544    275
26586    210
32305    179
        ... 
29337      1
27290      1
40998      1
20271      1
40964      1
Name: total_income, Length: 15399, dtype: int64

Taking a closer look at total_income outliers we see multiple oulier values.
Therefore we move on to group them. 

In [40]:
#Here we group by below average, average and above average, as following:
def income_group(total_income):
    if total_income < (credit['total_income'].mean()-credit['total_income'].std()):
        return 'below average'
    elif (credit['total_income'].mean()-credit['total_income'].std()) <= total_income <= (credit['total_income'].mean()+credit['total_income'].std()):
        return 'average'
    else:
        return 'above average'


credit['total_income_group'] = credit['total_income'].apply(income_group)
credit['total_income_group'].value_counts()

average          17908
above average     2332
below average     1284
Name: total_income_group, dtype: int64

Nicely done!! Next..

In [41]:
#First we categorized outliers now we move on.
#We aim to create a new DataFrame.
#We will only keep columns required for the purpose of this project analysis.
credit_2 = credit[['children', 'family_status', 'income_type', 'total_income','total_income_group', 'purpose', 'debt']].copy()
credit_2

,children,family_status,income_type,total_income,total_income_group,purpose,debt
0,1,married,employee,40620,average,real estate,0
1,1,married,employee,17932,average,car,0
2,0,married,employee,23341,average,real estate,0
3,3,married,employee,42820,above average,education,0
4,0,civil partnership,retiree,25378,average,wedding,0
...,...,...,...,...,...,...,...
21520,1,civil partnership,business,35966,average,real estate,0
21521,0,married,retiree,24959,average,car,0
21522,1,civil partnership,employee,14347,average,real estate,1
21523,3,married,employee,39054,average,car,1


#### Conclusion

Super! We have created a preprocessed dataset in a structure targeted for the project analysis requirements.  

## Answer these questions

### Is there a relation between having kids and repaying a loan on time?

In [42]:
credit_2['debt'].mean()
#First we find the mean of debt (i.e. loan not paid) to use for all following calculations

0.08088645233228024

We find that **debt** value ~0.08 indicates that most borrowers repay their loans.

In [43]:
#To answer this question we investigate the correlation between number of children and the mean of debt
credit_2.groupby('children')['debt'].mean()

children
0    0.075295
1    0.091470
2    0.094404
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

In [44]:
#Since result for 5 children is 0.00 we wish to re-check the value count of children.
credit_2['children'].value_counts()

0    14224
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

In [45]:
#And in percentages now.
credit_2['children'].value_counts(normalize=True)

0    0.660844
1    0.226027
2    0.095475
3    0.015332
4    0.001905
5    0.000418
Name: children, dtype: float64

Results show a direct correlation between having children vs. not having children and the likelihood to return debt.
Having no children makes you more likely to repay loan on time. This could be explained given their family expenses are relatively lower than families with children.
As we can see the average debt percentage of each group is close to the mean of debt.
Borrowers with no children are doing slightly better then those with children, but not significantly much better. 
Surprisingly we find that borrowers with 5 children absolutely never failed to return a loan. This might be an exception, or a coincidence given there are only 9 entries with 5 children in the dataset, accumulating to an insignificant 0.04%.
Those with 4 children are the most likely to fail to repay their loan. Probably since their family expenses are high.

**Yes, there is a relation between having kids and repaying a loan on time.
Having children makes you less likely to repay your load.**

### Is there a relation between marital status and repaying a loan on time?

In [46]:
#Again, we check correlation between family_status and debt mean we found previously. 
credit_2.groupby('family_status')['debt'].mean()

family_status
civil partnership    0.092912
divorced             0.071130
married              0.075202
unmarried            0.097405
widow / widower      0.065625
Name: debt, dtype: float64

In [47]:
#We wish to see the percentage of customers per category. 
credit_2['family_status'].value_counts(normalize=True)

married              0.575172
civil partnership    0.194016
unmarried            0.130691
divorced             0.055519
widow / widower      0.044601
Name: family_status, dtype: float64

We could come up with an explanation to this correlation.
Those who are married or were married (currently divorced or widowed) are more likely to repay their loans. We could assume these people are more reliable and responsible.
On the other hand, people in civil partnerships and unmarried people are less likely to repay their loan, possibly this is also reflected by their personal life choices such as not committing to marital relationships. 

**Yes, there is a correlation between marital status and repaying a loan on time.
Never married are less likely to reply their loans.**

### Is there a relation between income level and repaying a loan on time?

In [48]:
#First we check income level groups.
credit_2.groupby('total_income_group')['debt'].mean()

total_income_group
above average    0.069897
average          0.083091
below average    0.070093
Name: debt, dtype: float64

In [49]:
#Just out of curiosity, we further investigated the following:
#1) Total income group counts.
credit_2['total_income_group'].value_counts()

average          17908
above average     2332
below average     1284
Name: total_income_group, dtype: int64

In [50]:
#And in percentage.
credit_2['total_income_group'].value_counts(normalize=True)

average          0.832001
above average    0.108344
below average    0.059654
Name: total_income_group, dtype: float64

Results show a reasonable distribution, most are average whereas some are above and some are below.

In [51]:
#2) Income levels in relation to income type. 
income_pivot = credit_2.pivot_table(index='total_income_group', columns='income_type', values='debt', aggfunc='mean')
income_pivot

income_type,business,civil servant,employee,retiree
total_income_group,,,,
above average,0.062833,0.034884,0.088265,0.044643
average,0.076847,0.063953,0.096778,0.056869
below average,0.060000,0.036145,0.088481,0.055777


In the above pivot table we see that in all income types the distribution is alike.

In [52]:
#3) Lastly we performed the income type value counts.
credit_2['income_type'].value_counts()

employee         11123
business          5086
retiree           3856
civil servant     1459
Name: income_type, dtype: int64

In [53]:
#And in percentage.
credit_2['income_type'].value_counts(normalize=True)

employee         0.516772
business         0.236294
retiree          0.179149
civil servant    0.067785
Name: income_type, dtype: float64

As we see in the pivot table, retirees tend to fail less then others.
People with average income in all groups are more likely to fail more than the others in same income type group with above or less than average income.
The least risky loans are to civil servants, those with above or below average.


**Yes, we can conclude that there is a relation between income level and repaying a loan on time. An average income predicts less likelihood to repay loan, whereas below or above average imply high likelihood to replay in each group.**
We assume that people with below average income are more calculated and responsible with money in their day to day lives, and hence they better manage repaying loans.
People with above average income, obviously have resources to repay the loan on time.
The majority with average income might live a higher standard of living than their actual ability and therefore have difficulties repaying their loan in time.

### How do different loan purposes affect on-time repayment of the loan?

In [54]:
#We calculate the mean debt per purpose values.
credit_2.groupby('purpose')['debt'].mean()

purpose
car            0.093395
education      0.091994
real estate    0.072147
wedding        0.079216
Name: debt, dtype: float64

Loans for car and education are less likely to be repaid on time. Unlike wedding and real estate purposes that are more likely to be returned on time.
We can assume this is due to the fact that usually real estate and wedding loans are taken by couples sharing the responsibility and possibly having two salaries making it easier to repay the loan. Car and education can be solitary investments and hence making it challenging on the individual to repay on time. 

In [55]:
#Just to get a better understanding of the purpose values distribution
#we investigated the distribution among the various purposes.
credit_2['purpose'].value_counts()

real estate    10839
car             4315
education       4022
wedding         2348
Name: purpose, dtype: int64

In [56]:
#And again looking at the percentages.
credit_2['purpose'].value_counts(normalize=True)

real estate    0.503577
car            0.200474
education      0.186861
wedding        0.109088
Name: purpose, dtype: float64

In [57]:
credit_2.groupby('purpose')['debt'].count()

purpose
car             4315
education       4022
real estate    10839
wedding         2348
Name: debt, dtype: int64

In [58]:
credit_2.groupby('purpose')['debt'].sum()

purpose
car            403
education      370
real estate    782
wedding        186
Name: debt, dtype: int64

In [59]:
#We build a pivot table using pivot_table() to calculate not only the ratio, but count and sum for each category.
credit_2['value']=1
credit_2.pivot_table(index='purpose', columns='debt', values='value', aggfunc=['sum','count'], margins=True)

sum               count             
debt             0     1    All      0     1    All
purpose                                            
car           3912   403   4315   3912   403   4315
education     3652   370   4022   3652   370   4022
real estate  10057   782  10839  10057   782  10839
wedding       2162   186   2348   2162   186   2348
All          19783  1741  21524  19783  1741  21524

Okay!! We successfully built the pivot table using pivot_table() to calculate the count and sum for each category.
We used also group-by and pivot_table just to be on the safe side :-)
We can conclude that real estate has the highest values for debt / no debt since by far there are more loans taken for this purpose in the dataset, approx. 50% in total.
Most real estate loans were repaid, whereas only approx. 7% not.
For car and education approx. 10% were not repaid on time.
And for weddings approx. 8% of the loans were not repaid on time.

**Here we conclude by stating that the purpose of loan implies what is the chance of it being repaid on time.**
Loans for cars are the least likely to be repaid on time.
Loans for eduction are second less likely to be repaid on time.
Loans for weddings come third.
And loans for real estate are the most likely to be repaid on time.

## General conclusion

Our project was to prepare guidance for a bank's loan division. Specifically we aimed to estimate the credit score for potential customers based on provided data on former  customers' credit worthiness.

**Here we are concise in describing the general and final conclusions that would be provided to the bank division.**

In order to draw conclusions we opened the data file, preprocessed the data for missing values, fixed data types, deleted duplicated data and finally categorized the data.
Next, we addressed the questions asked, and reached our final conclusions as following:

- There is a correlation between having children and not having children, and repaying the loan on time. **Having no children increases the likelihood to repay**, yet only by approx. 2% which is not significant.

- There is a correlation between marital status and repaying the loan on time. **Having been married, and if we include also those who are currently divorced or widowed, accumulate to  over 60% of the dataset and significantly predict higher likelihood to repay a loan on time.**

- There is a correlation between income level and repaying the loan on time. **Having an average income by far, about 83%, indicates low likelihood to repay a loan on time.**

- There is a correlation between purpose and repaying the loan on time. **The highest return of loans are loans for real estate 50%, and next additional 10% for weddings. Whereas loans for the purpose of education follows and car comes last.**

We have closely followed the project definition and kept the project structure as provided. However, in a real life scenario, prior to providing the bank with the conclusions drawn, we would recommend to further investigate the additional columns and how other factors, such as gender, age, days employed, and customers' education level might impact customers' on time loan return.